In [301]:
import librosa
# import numpy as np
import requests
import urllib3
import jsonpickle
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


save_path = "data"
sample_rate = 16000
debug = True
session_id = "test_session"
timestamp = 0

In [35]:
# load audio file
audio_en, sr = librosa.load(f"{save_path}/test_en.wav", sr=sample_rate)
assert sr == sample_rate
audio_cs, sr = librosa.load(f"{save_path}/test_cs.wav", sr=sample_rate)
assert sr == sample_rate

if debug:
    print(type(audio_en), audio_en.shape)
    print(type(audio_cs), audio_cs.shape)

<class 'numpy.ndarray'> (3270529,)
<class 'numpy.ndarray'> (3956943,)


In [308]:
def crate_session():
    try:
        response = requests.get(
            f"https://slt.ufal.mff.cuni.cz:5003/create_session?session_id={session_id}",
            verify=False,
        )
        print(response.text)
    except Exception as e:
        print(e)


def end_session():
    try:
        response = requests.get(
            f"https://slt.ufal.mff.cuni.cz:5003/end_session?session_id={session_id}", verify=False
        )
        print(response.text)
    except Exception as e:
        print(e)


def get_active_sessions():
    try:
        response = requests.get(
            "https://slt.ufal.mff.cuni.cz:5003/get_active_sessions", verify=False
        )
        print(response.text)
    except Exception as e:
        print(e)


def audio_to_dict(audio):
    return {str(index): value for index, value in enumerate(audio.tolist())}


def submit_audio():
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/submit_audio_chunk?session_id={session_id}",
            verify=False,
            json={
                "timestamp": timestamp,
                "chunk": audio_to_dict(audio=audio_en[:10]),
            },  # 10 seconds
        )
        print(response.text)
    except Exception as e:
        print(e)


def offload_computation_get():
    try:
        response = requests.get(
            "https://slt.ufal.mff.cuni.cz:5003/offload_ASR",
            verify=False,
        )
        print(response.text)
    except Exception as e:
        print(e)


def offload_computation_post():
    try:
        response = requests.post(
            "https://slt.ufal.mff.cuni.cz:5003/offload_ASR",
            verify=False,
            json={
                "session_id": session_id,
                "timestamp": timestamp,
                "tsw": [
                    (29.0, 29.16, " That's"),
                    (29.16, 29.28, " about"),
                    (29.28, 29.5, " four"),
                    (29.5, 29.9, " seconds."),
                ],
                "ends": [5.44, 9.16, 11.78, 17.26, 23.9, 26.2, 29.9],
                "language": "en",
            },
        )
        print(response.text)
    except Exception as e:
        print(e)


def get_latest_text_chunk_versions():
    try:
        response = requests.get(
            f"https://slt.ufal.mff.cuni.cz:5003/get_latest_text_chunk_versions?session_id={session_id}&language=en",
            verify=False,
        )
        print(response.text)
    except Exception as e:
        print(e)


def post_latest_text_chunks(versions={}):
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/get_latest_text_chunks?session_id={session_id}&language=en",
            verify=False,
            json={"versions": versions},
        )
        print(response.text)
    except Exception as e:
        print(e)


def edit_asr_chunk(timestamp=0, version=0, new_text: str = ""):
    # request_data = request.get_json()
    # session_id = request.args.get("session_id", default=None, type=str)
    # language = request.args.get("language", default=None, type=str)
    # timestamp = request_data["timestamp"]
    # version = request_data["version"]
    # text = request_data["text"]

    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/edit_asr_chunk?session_id={session_id}&language=en",
            verify=False,
            json={"timestamp": timestamp, "version": version, "text": new_text},
        )
        print(response.text)
    except Exception as e:
        print(e)


def rate_text_chunk(timestamp=0, version=0, rating_update=1):
    # session_id = request.args.get("session_id", default=None, type=str)
    # language = request.args.get("language", default=None, type=str)

    # timestamp = request_data["timestamp"]
    # version = request_data["version"]
    # rating_update = request_data["rating_update"]
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/rate_text_chunk?session_id={session_id}&language=en",
            verify=False,
            json={"timestamp": timestamp, "version": version, "rating_update": rating_update},
        )
        print(response.text)
    except Exception as e:
        print(e)


def get_correction_rules():
    try:
        response = requests.get(
            f"https://slt.ufal.mff.cuni.cz:5003/get_correction_rules?session_id={session_id}&language=en",
            verify=False,
        )
        print(response.text)
    except Exception as e:
        print(e)


def submit_correction_rules():
    # rules have the following general structure:
    # [
    #     {
    #         "source_strings": [
    #             {
    #                 "string": "str",
    #                 "active": "bool"
    #             },
    #         ],
    #         "to": "str",
    #         "version": "int",
    #     },
    # ]
    rules = [
        {
            "source_strings": [
                {
                    "string": "four",
                    "active": True,
                },
            ],
            "to": "five",
            "version": 0,
        },
        {
            "source_strings": [
                {
                    "string": "five",
                    "active": True,
                },
            ],
            "to": "six",
            "version": 0,
        },
    ]
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/submit_correction_rules?session_id={session_id}&language=en",
            verify=False,
            json=rules,
        )
        print(response.text)
    except Exception as e:
        print(e)


def switch_source_language(language: str = "en"):
    # request_data = request.get_json()
    # session_id = request.args.get("session_id", default=None, type=str)
    # language = request_data["language"]
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/switch_source_language?session_id={session_id}",
            verify=False,
            json={"language": language},
        )
        print(response.text)
    except Exception as e:
        print(e)


def switch_transcript_language(language: str = "en"):
    # request_data = request.get_json()
    # session_id = request.args.get("session_id", default=None, type=str)
    # language = request_data["language"]
    try:
        response = requests.post(
            f"https://slt.ufal.mff.cuni.cz:5003/switch_transcript_language?session_id={session_id}",
            verify=False,
            json={"language": language},
        )
        print(response.text)
    except Exception as e:
        print(e)

In [325]:
#  TEST session creation and deletion
end_session()


{"success": true, "message": "Successfully ended session test_session"}


In [309]:
crate_session()
get_active_sessions()
timestamp = 0

{"success": true, "message": "Successfully created session test_session"}
{"active_sessions": ["test_session"]}


{"success": true, "session_id": "test_session"}


In [311]:
submit_audio()
offload_computation_get()
offload_computation_post()
timestamp += 1

{"success": true, "session_id": "test_session"}
{
    "session_id": "test_session",
    "timestamp": 1,
    "source_language": "en",
    "transcript_language": "en",
    "prompt": "",
    "audio": [
        8.939230156101985e-07,
        1.730002509248152e-06,
        -3.0721382699994138e-06,
        -1.2614202660188312e-07,
        -1.0721962553361664e-06,
        -2.666301725184894e-06,
        3.994047801825218e-06,
        -6.392704108293401e-07,
        9.277324579670676e-07,
        5.490992407430895e-06,
        8.939230156101985e-07,
        1.730002509248152e-06,
        -3.0721382699994138e-06,
        -1.2614202660188312e-07,
        -1.0721962553361664e-06,
        -2.666301725184894e-06,
        3.994047801825218e-06,
        -6.392704108293401e-07,
        9.277324579670676e-07,
        5.490992407430895e-06
    ]
}
{
    "success": true
}


In [312]:
get_latest_text_chunk_versions()
post_latest_text_chunks({})

{"success": true, "session_id": "test_session", "versions": {"0": 0}}
{"success": true, "session_id": "test_session", "text_chunks": [{"timestamp": 0, "version": 0, "text": " That's about four seconds."}], "versions": {"0": 0}}


In [284]:
edit_asr_chunk(timestamp=0, version=0, new_text="This is a four edit.")

{"success": true, "session_id": "test_session", "text": "This is a five edit.", "timestamp": 0, "version": 1}


In [285]:
edit_asr_chunk(timestamp=0, version=0, new_text="This is five edit.")

{"success": true, "session_id": "test_session", "text": "This is six edit.", "timestamp": 0, "version": 2}


In [246]:
rate_text_chunk(timestamp=0,version=0,rating_update=1)

{"success": true, "message": "Successfully updated rating for test_session, language en, chunk_id 0, chunk_version 0, rating_update 1, new_rating 2"}


In [288]:
submit_correction_rules()
get_correction_rules()

{"success": true, "message": "Successfully uploaded rules for session test_session, language en"}
[{"source_strings": [{"string": "four", "active": true}], "to": "five", "version": 0}, {"source_strings": [{"string": "five", "active": true}], "to": "six", "version": 0}]


In [295]:
switch_source_language("cs")
switch_transcript_language("cs")

{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session"}


In [313]:
def integration_test():
    crate_session()

    def submit_audio_chunk(timestamp, audio):
        try:
            response = requests.post(
                f"https://slt.ufal.mff.cuni.cz:5003/submit_audio_chunk?session_id={session_id}",
                verify=False,
                json={
                    "timestamp": timestamp,
                    "chunk": audio_to_dict(
                        audio=audio_en[timestamp * sample_rate : (timestamp + 1) * sample_rate]
                    ),
                },  # 10 seconds
            )
            print(response.text)
        except Exception as e:
            print(e)

    for timestamp in range(0, 60):
        submit_audio_chunk(timestamp, audio_en)
        get_latest_text_chunk_versions()
        post_latest_text_chunks({})
        time.sleep(2)

In [324]:
integration_test()

{"success": true, "message": "Successfully created session test_session"}
{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session", "versions": {}}
{"success": true, "session_id": "test_session", "text_chunks": [], "versions": {}}
{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session", "versions": {}}
{"success": true, "session_id": "test_session", "text_chunks": [], "versions": {}}
{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session", "versions": {}}
{"success": true, "session_id": "test_session", "text_chunks": [], "versions": {}}
{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session", "versions": {}}
{"success": true, "session_id": "test_session", "text_chunks": [], "versions": {}}
{"success": true, "session_id": "test_session"}
{"success": true, "session_id": "test_session", "versions": {}}
{"success": true, "session_id": "t